In [ ]:
import os
os.environ['CALITP_BUCKET__GTFS_SCHEDULE_RAW'] = 'gs://test-calitp-gtfs-schedule-raw-v2'
from calitp.storage import get_fs
from google.cloud import storage

fs = get_fs()
PARTITIONED_ARTIFACT_METADATA_KEY = "PARTITIONED_ARTIFACT_METADATA"
client = storage.Client(project="cal-itp-data-infra")
fs, client

In [ ]:
old_v2_bucket = storage.Bucket(client=client, name="test-calitp-gtfs-schedule-raw")
old_v2_outcomes = list(old_v2_bucket.list_blobs(prefix="download_schedule_feed_results"))

new_v2_bucket = storage.Bucket(client=client, name=os.environ['CALITP_BUCKET__GTFS_SCHEDULE_RAW'].replace('gs://', ''))
new_v2_outcomes = list(new_v2_bucket.list_blobs(prefix="download_schedule_feed_results"))

len(old_v2_outcomes), old_v2_outcomes[-1], len(new_v2_outcomes), new_v2_outcomes[0]

In [ ]:
one_results_blob = old_v2_outcomes[0]
one_results_blob.bucket.name, one_results_blob.path, one_results_blob.name

In [ ]:
import json
from collections import Counter

Counter(
    str(json.loads(file.metadata[PARTITIONED_ARTIFACT_METADATA_KEY]).keys())
    for file in old_v2_outcomes
), Counter(
    str(json.loads(file.metadata[PARTITIONED_ARTIFACT_METADATA_KEY]).keys())
    for file in new_v2_outcomes
)

In [ ]:
with fs.open(f"gs://{one_results_blob.bucket.name}/{one_results_blob.name}") as f:
    results = [json.loads(line) for line in f.readlines()]
results[0]

In [ ]:
import base64
from typing import ClassVar, List, Optional

import pendulum
from calitp.storage import (
    GTFSDownloadConfig,
    GTFSFeedType,
    GTFSScheduleFeedExtract,
    PartitionedGCSArtifact,
    ProcessingOutcome,
)
from google.cloud import storage
from pydantic import HttpUrl, parse_obj_as
from tqdm.auto import tqdm

class GTFSDownloadOutcome(ProcessingOutcome):
    config: GTFSDownloadConfig
    extract: Optional[GTFSScheduleFeedExtract]


class DownloadFeedsResult(PartitionedGCSArtifact):
    bucket: ClassVar[str] = "gs://test-calitp-gtfs-schedule-raw-v2"
    table: ClassVar[str] = "download_schedule_feed_results"
    partition_names: ClassVar[List[str]] = ["dt", "ts"]
    ts: pendulum.DateTime
    end: pendulum.DateTime
    outcomes: List[GTFSDownloadOutcome]
    @property
    def dt(self) -> pendulum.Date:
        return self.ts.date()

def handle_one_results_blob(results_blob, dry_run=True):
    to_copy = []
    new_outcomes = []
    with fs.open(f"gs://{results_blob.bucket.name}/{results_blob.name}") as f:
        old_outcomes = [json.loads(line) for line in f.readlines()]
    for result in tqdm(old_outcomes):
        old_extract = result["extract"]
        old_airtable_record = result["airtable_record"]
        
        # we always need to be able to get a download config, even if there's no "real" file underlying it
        new_config = GTFSDownloadConfig(
            extracted_at=None,
            name=old_airtable_record["name"],
            url=parse_obj_as(HttpUrl, old_airtable_record["pipeline_url"]),
            feed_type=GTFSFeedType.schedule,
            schedule_url_for_validation=None,
            auth_query_params={},
            auth_headers={},
        )
        
        # if we had a failure, there is no extract to copy
        if not result['extract']:
            new_outcomes.append(
                GTFSDownloadOutcome(
                    success=result["success"],
                    exception=Exception(result["exception"]),
                    config=new_config,
                    extract=None,
                )
            )
            continue
        
        # if we were successful, we should have a 
        ts = pendulum.parse(old_extract["ts"])
        dt = ts.date()
        base64_url = base64.urlsafe_b64encode(
            old_airtable_record["pipeline_url"].encode()
        ).decode()

        # the old v2 files have url then ts, but we will be swapping them
        old_blob_key = f"schedule/dt={dt.to_date_string()}/base64_url={base64_url}/ts={ts.to_iso8601_string()}/{old_extract['filename']}"
        old_blob = results_blob.bucket.get_blob(old_blob_key)
        if old_blob is None:
            #print(new_config.url)
            new_outcomes.append(
                GTFSDownloadOutcome(
                    success=False,
                    exception=Exception("blob missing during backfill operation"),
                    config=new_config,
                    extract=None,
                )
            )
            continue

        assert old_blob.metadata[PARTITIONED_ARTIFACT_METADATA_KEY] == json.dumps(old_extract)

        new_extract = GTFSScheduleFeedExtract(
            filename=old_extract['filename'],
            config=new_config,
            response_code=old_extract["response_code"],
            response_headers=old_extract["response_headers"],
            ts=old_extract["ts"],
        )
        
        to_copy.append((f"gs://{old_blob.bucket.name}/{old_blob.name}", new_extract))
        new_outcomes.append(
            GTFSDownloadOutcome(
                success=result["success"],
                exception=result["exception"],
                config=new_config,
                extract=new_extract,
            )
        )

    # only copy results once successful
    old_metadata = json.loads(results_blob.metadata[PARTITIONED_ARTIFACT_METADATA_KEY])
    new_result = DownloadFeedsResult(
        ts=pendulum.parse(old_metadata['ts']),
        end=pendulum.parse(old_metadata['end']),
        outcomes=new_outcomes,
        filename=old_metadata['filename'],
    )
    assert len(old_outcomes) == len(new_outcomes)
    assert len(to_copy) == len([result for result in new_result.outcomes if result.success])
    if not dry_run:
        pass
    return new_result, to_copy

In [ ]:
new_result, to_copy = handle_one_results_blob(results_blob=one_results_blob, dest_bucket=None)
len(to_copy), to_copy[0], new_result.bucket, new_result.path

In [ ]:
to_copy[0][0], to_copy[0][1].path

In [ ]:
results = []
to_copies = []

for results_blob in tqdm(old_v2_outcomes):
    new_result, to_copy = handle_one_results_blob(results_blob=one_results_blob)
    results.append(new_result)
    to_copies.extend(to_copy)
len(results), len(to_copies)

In [ ]:
import pandas as pd

df = pd.DataFrame(first_files, columns=["blob"])
df["metadata"] = df.blob.apply(
    lambda b: str(json.loads(b.metadata[PARTITIONED_ARTIFACT_METADATA_KEY]).keys())
)
df.head()

In [ ]:
import gzip
import os
from functools import lru_cache
from typing import Dict

import pendulum
from calitp.storage import (
    AirtableGTFSDataExtract,
    AirtableGTFSDataRecord,
    get_latest_file,
)

# os.environ["CALITP_BUCKET__AIRTABLE"] =


@lru_cache
def get_airtable_gtfs_records_for_day(
    dt: pendulum.Date,
) -> Dict[str, AirtableGTFSDataRecord]:
    file = get_latest_file(
        # AirtableGTFSDataExtract.bucket,
        "gs://test-calitp-airtable",
        AirtableGTFSDataExtract.table,
        prefix_partitions={
            "dt": dt,
        },
        partition_types={
            "ts": pendulum.DateTime,
        },
    )

    with get_fs().open(file.name, "rb") as f:
        content = gzip.decompress(f.read())
    records = [
        AirtableGTFSDataRecord(**json.loads(row))
        for row in content.decode().splitlines()
    ]

    return {record.id: record for record in records}


def schedule_record_ids_for_validation_to_actual_url(dt, record_ids):
    if not record_ids:
        return None
    if len(record_ids) == 1:
        records = get_airtable_gtfs_records_for_day(dt)
        return records[record_ids[0]].uri
    raise RuntimeError


len(get_airtable_gtfs_records_for_day(pendulum.Date(2022, 8, 15)))